<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/Voting_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
# 여기서는 기본 추정기의 파라미터를 튜닝하는 대신, 가중치만 조정합니다.

vr_params = {
'weights': [[1, 1], [1, 2], [2, 1]],
'voting': ['hard', 'soft']
}

# BaggingRegressor

bg_params = {
'n_estimators': [10, 20, 50, 100],
'max_features': [0.5, 0.7, 0.9, 1.0],
'max_features': [0.5, 0.7, 0.9, 1.0],
'bootstrap': [True, False],
'bootstrap_features': [True, False]
}

kr_params = {
'alpha': np.logspace(-4, 1, 20),
'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
'degree': [2, 3, 4],  # for poly kernel
'coef0': [0.0, 0.1, 0.5, 1.0],  # for poly and sigmoid kernels
'gamma': np.logspace(-4, 1, 20)  # for rbf, poly and sigmoid kernels
}

theilsen_params = {
'fit_intercept': [True, False],
'max_subpopulation': [5000, 10000, 20000],
'max_iter': [100, 300, 500, 1000],
'tol': [1e-5, 1e-4, 1e-3]
}

In [3]:
# Reduce Memory Usage
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [4]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import RandomizedSearchCV


# Load the data
train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [5]:
train_df_org_reduced = reduce_memory_usage(train_df_org)
test_df_reduced = reduce_memory_usage(test_df)

Memory usage of dataframe is 3.98 MB
Memory usage became:  2.38531494140625  MB
Memory usage of dataframe is 0.46 MB
Memory usage became:  0.27829742431640625  MB


In [6]:
from sklearn.metrics import make_scorer
from sklearn.exceptions import ConvergenceWarning
import warnings

def top_10_percent_recall(y_true, y_pred):
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('error', category=ConvergenceWarning)
            # 예측을 수행하는 대신 직접 받은 예측값을 사용

                        # 상위 10% 예측 및 상위 5% 실제값 기준 설정
            top_10_percent_cutoff = np.percentile(y_pred, 90)
            top_5_percent_cutoff = np.percentile(y_true, 95)
            top_10_pred_indices = np.where(y_pred >= top_10_percent_cutoff)[0]
            top_5_true_indices = np.where(y_true >= top_5_percent_cutoff)[0]

            # 교집합 계산
            common_indices = np.intersect1d(top_10_pred_indices, top_5_true_indices)

            # # 디버깅용 출력 (상위 10% 예측, 상위 5% 실제 값, 교집합)
            # print(f"Top 10% Predicted Indices: {top_10_pred_indices}")
            # print(f"Top 5% True Indices: {top_5_true_indices}")
            # print(f"Common Indices (Intersection): {common_indices}")

            # 실제 상위 5% 중 예측 상위 10%에 포함된 비율 (Recall 계산)
            if len(top_5_true_indices) == 0:
                return 0  # 상위 5% 데이터가 없으면 Recall 0
            recall = len(common_indices) / len(top_5_true_indices)

            return recall
    except Warning:
        return -np.inf  # 경고 발생 시 매우 낮은 점수 반환


# 사용자 정의 평가 함수로 스코어러 만들기
custom_scorer = make_scorer(top_10_percent_recall, greater_is_better=True)

In [7]:
# import numpy as np

# # 상위 10% 예측 Recall 계산 함수 (수정)
# def not_top_10_percent_recall(y_true, y_pred):
#     # 상위 10% 예측 및 상위 5% 실제값 기준 설정
#     top_10_percent_cutoff = np.percentile(y_pred, 90)
#     top_5_percent_cutoff = np.percentile(y_true, 95)

#     # 상위 10% 예측값과 상위 5% 실제값 추출
#     top_10_pred_indices = np.where(y_pred >= top_10_percent_cutoff)[0]
#     top_5_true_indices = np.where(y_true >= top_5_percent_cutoff)[0]

#     # 교집합 계산
#     common_indices = np.intersect1d(top_10_pred_indices, top_5_true_indices)

#     # 디버깅용 출력 (상위 10% 예측, 상위 5% 실제 값, 교집합)
#     print(f"Top 10% Predicted Indices: {top_10_pred_indices}")
#     print(f"Top 5% True Indices: {top_5_true_indices}")
#     print(f"Common Indices (Intersection): {common_indices}")

#     # 실제 상위 5% 중 예측 상위 10%에 포함된 비율 (Recall 계산)
#     if len(top_5_true_indices) == 0:
#         return 0  # 상위 5% 데이터가 없으면 Recall 0
#     recall = len(common_indices) / len(top_5_true_indices)

#     return recall

# custom_scorer2 = make_scorer(not_top_10_percent_recall, greater_is_better=True)

In [8]:

def val_score(model, valiation_df):
  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거
  X = valiation_df.iloc[:, 1:-1]
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=model.predict(X)
  id_y['y_pred']=y_pred

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  return top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df)

In [9]:
correlation_matrix = train_df_org_reduced.iloc[:, 1:].corr()

# 타겟 변수 'y'와의 상관관계를 계산하고, 상관관계가 0.5 이상인 특성을 추출합니다.
high_correlation_features = correlation_matrix['y'][abs(correlation_matrix['y']) > 0.5].index.tolist()

# 타겟 변수 'y'를 제외하고 특성 리스트에서 선택
high_correlation_features.remove('y')

high_correlation_features

['x_0', 'x_1', 'x_3', 'x_4', 'x_5', 'x_7', 'x_8', 'x_9', 'x_10']

In [10]:
# df_shuffled= train_df_org
df_shuffled= train_df_org_reduced

# 균등하게 나누기 위해 임의로 데이터를 8:2로 분할
split_index = int(len(df_shuffled) * 0.8)
train_df, valiation_df = df_shuffled[:split_index], df_shuffled[split_index:]


In [11]:
X = train_df[high_correlation_features]
y = train_df['y']
X_test=valiation_df[high_correlation_features]
y_test=valiation_df['y']

In [12]:
from ast import mod
import joblib

models=[]
theilsen_model=joblib.load('/content/drive/MyDrive/dacon/model/feature_model/theilsen_model.pkl')
kr_model=joblib.load('/content/drive/MyDrive/dacon/model/feature_model/kr_model.pkl')
models.append(theilsen_model)
models.append(kr_model)
catboost_model=joblib.load('/content/drive/MyDrive/dacon/model/feature_model/catboost_model.pkl')
models.append(catboost_model)
xgb_model=joblib.load('/content/drive/MyDrive/dacon/model/feature_model/xgb_model.pkl')
models.append(xgb_model)

for i in models:
  y_pred=i.predict(X_test)
  print(i,top_10_percent_recall(y_test, y_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TheilSenRegressor from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KernelRidge from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


TheilSenRegressor(fit_intercept=False, max_subpopulation=10000) 0.8669950738916257
KernelRidge(alpha=0.00206913808111479, coef0=0.1, degree=2,
            gamma=0.0003359818286283781, kernel='sigmoid') 0.8719211822660099
<catboost.core.CatBoostRegressor object at 0x7a1b8a0eaaa0> 0.8719211822660099
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None,
             num_par

In [18]:
model_736=joblib.load('/content/drive/MyDrive/dacon/model/Reduce_KernelRidge_model.pkl')
test_df_reduced


n_X_test=valiation_df.iloc[:,1:-1]
n_y_test=valiation_df['y']

y_pred=model_736.predict(n_X_test)

print(model_736,top_10_percent_recall(n_y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KernelRidge from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


KernelRidge(alpha=84.89148242660839, coef0=0.4434590423297464, degree=4,
            gamma=5.227428293819941, kernel='poly') 0.8793103448275862


In [14]:
valiation_df.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,y
32094,TRAIN_32094,1.006836,-2.013672,1.228516,0.905273,-0.259766,-1.786133,0.468506,-0.145874,0.625000,0.313965,0.077881,82.6875
32095,TRAIN_32095,1.111328,-2.494141,1.207031,0.872559,-0.250244,-1.811523,0.493896,-0.180298,0.676270,0.264648,0.095581,85.1875
32096,TRAIN_32096,1.043945,-2.503906,1.198242,0.790527,-0.264404,-1.791016,0.495117,-0.190674,0.688965,0.254639,0.093628,82.4375
32097,TRAIN_32097,1.073242,-2.484375,1.201172,0.844238,-0.264404,-1.799805,0.500000,-0.192505,0.684082,0.257812,0.095032,84.0000
32098,TRAIN_32098,1.083984,-2.439453,1.209961,0.902344,-0.250977,-1.816406,0.481689,-0.175537,0.666992,0.280762,0.095764,84.7500


In [12]:
from sklearn.linear_model import TheilSenRegressor
from sklearn.metrics import mean_squared_error

model = TheilSenRegressor()


# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=theilsen_params,
    n_iter=30,  # 시도할 하이퍼파라미터 조합의 수
    cv=3,  # 3-fold cross-validation
    scoring=custom_scorer,  # 평가 척도
    random_state=42,  # 결과 재현을 위한 랜덤 시드
    n_jobs=-1,  # 병렬처리
    verbose=2  # 진행 상황 출력
)

# 모델 훈련 및 하이퍼파라미터 튜닝
random_search.fit(X, y)

# 최적의 모델 및 하이퍼파라미터 확인
theilsen_model = random_search.best_estimator_
theilsen_params = random_search.best_params_

y_pred = theilsen_model.predict(X_test)

# RMSE 계산
recall = top_10_percent_recall(y_test, y_pred)
print("TheilSenRegressor:" , recall)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Top 10% Predicted Indices: [   9   12   16   49   50   53   57   58   68   72   73   76   91   93
   95  100  107  109  131  135  138  167  172  181  196  206  208  230
  232  241  248  262  298  302  308  335  355  373  398  405  426  427
  430  438  453  460  480  489  493  532  547  578  582  584  604  605
  617  654  675  694  700  713  735  740  751  752  777  781  783  788
  791  802  803  805  809  819  837  839  843  854  861  862  867  868
  872  877  886  924  930  937  949  959  963  964  977  980 1010 1027
 1033 1052 1057 1063 1074 1078 1079 1084 1087 1093 1097 1112 1113 1127
 1142 1154 1165 1168 1170 1171 1181 1196 1202 1209 1227 1234 1236 1239
 1240 1248 1281 1286 1290 1301 1304 1316 1329 1339 1350 1352 1363 1366
 1369 1372 1375 1379 1396 1426 1439 1441 1447 1452 1472 1476 1490 1506
 1509 1523 1558 1559 1560 1561 1567 1568 1579 1585 1596 1607 1610 1629
 1644 1645 1655 1698 1714 1740 1751 1758 1776 1783 1786 1805 1806 1826
 1842 1850 1863 1883 1898 1941 1946 1950 1953 1957

In [13]:



# 메모리가 터지네..
from sklearn.kernel_ridge import KernelRidge

model = KernelRidge()


# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=kr_params,
    n_iter=30,  # 시도할 하이퍼파라미터 조합의 수
    cv=3,  # 3-fold cross-validation
    scoring=custom_scorer,  # 평가 척도
    random_state=42,  # 결과 재현을 위한 랜덤 시드
    # n_jobs=-1,  # 병렬처리
    verbose=2  # 진행 상황 출력
)

# 모델 훈련 및 하이퍼파라미터 튜닝
random_search.fit(X, y)

# 최적의 모델 및 하이퍼파라미터 확인
kr_model = random_search.best_estimator_
kr_params = random_search.best_params_

y_pred = kr_model.predict(X_test)

# RMSE 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)


print("KernelRidge:" , rmse)

Streaming output truncated to the last 5000 lines.
[CV] END alpha=2.9763514416313193, coef0=0.5, degree=2, gamma=0.14384498882876628, kernel=poly; total time=  48.3s
Top 10% Predicted Indices: [    9    43    56 ... 10661 10668 10685]
Top 5% True Indices: [    9    38    56    64    67   114   124   143   156   189   198   209
   229   331   343   361   389   391   400   412   442   468   473   478
   484   490   497   528   535   545   566   575   577   628   637   673
   689   699   723   728   763   786   792   807   835   864   877   878
   881   885   913   937   943   969   988   997  1029  1032  1044  1059
  1108  1112  1113  1133  1200  1215  1279  1281  1287  1316  1329  1342
  1348  1360  1367  1382  1388  1396  1402  1426  1445  1449  1457  1464
  1506  1560  1614  1626  1639  1669  1715  1726  1749  1752  1754  1756
  1762  1769  1779  1803  1819  1820  1832  1846  1878  1896  1905  1956
  1960  1980  1993  1998  2011  2017  2035  2043  2071  2090  2121  2128
  2137  2151  

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Top 10% Predicted Indices: [    6    17    26 ... 10666 10675 10680]
Top 5% True Indices: [    6    17    26    92   105   212   231   310   319   324   362   402
   411   419   427   429   451   467   475   494   503   516   555   598
   610   674   693   698   714   719   742   813   820   826   835   857
   878   905   948   949   967   974   977   997  1037  1096  1117  1151
  1157  1159  1168  1194  1208  1221  1253  1254  1273  1281  1284  1296
  1299  1387  1395  1397  1421  1423  1443  1475  1512  1522  1549  1562
  1566  1596  1631  1684  1687  1748  1753  1768  1792  1802  1821  1904
  1923  1935  1968  1982  1989  2004  2064  2066  2071  2098  2111  2120
  2129  2132  2242  2251  2302  2322  2398  2403  2432  2452  2467  2471
  2483  2502  2541  2542  2550  2556  2588  2599  2612  2631  2663  2701
  2733  2747  2766  2783  2823  2833  2843  2868  2873  2931  2934  2965
  2986  3001  3020  3031  3062  3083  3087  3110  3162  3191  3229  3235
  3238  3257  3287  3318  3324  33

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Top 10% Predicted Indices: [    2    13    32 ... 10655 10657 10694]
Top 5% True Indices: [   13    43    46    61    65   112   120   124   144   181   229   292
   312   323   382   384   409   431   448   462   521   531   532   543
   564   565   591   615   666   668   681   682   692   725   730   733
   746   806   810   821   825   856   862   920   934   959   962   989
  1036  1037  1062  1070  1079  1081  1114  1115  1117  1127  1152  1192
  1203  1213  1239  1256  1299  1366  1368  1384  1419  1422  1538  1582
  1594  1621  1625  1627  1649  1656  1684  1710  1744  1750  1777  1790
  1807  1827  1839  1855  1867  1875  1880  1905  1909  1926  1941  1945
  1950  1954  1997  2007  2012  2074  2103  2121  2133  2230  2235  2245
  2249  2256  2258  2280  2284  2292  2334  2350  2356  2384  2420  2450
  2477  2492  2506  2508  2509  2511  2517  2530  2574  2597  2608  2630
  2643  2655  2703  2707  2819  2869  2881  2923  2933  2944  2980  3014
  3026  3030  3043  3060  3101  31

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:239: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(


Top 10% Predicted Indices: [    9    43    56 ... 10661 10668 10685]
Top 5% True Indices: [    9    38    56    64    67   114   124   143   156   189   198   209
   229   331   343   361   389   391   400   412   442   468   473   478
   484   490   497   528   535   545   566   575   577   628   637   673
   689   699   723   728   763   786   792   807   835   864   877   878
   881   885   913   937   943   969   988   997  1029  1032  1044  1059
  1108  1112  1113  1133  1200  1215  1279  1281  1287  1316  1329  1342
  1348  1360  1367  1382  1388  1396  1402  1426  1445  1449  1457  1464
  1506  1560  1614  1626  1639  1669  1715  1726  1749  1752  1754  1756
  1762  1769  1779  1803  1819  1820  1832  1846  1878  1896  1905  1956
  1960  1980  1993  1998  2011  2017  2035  2043  2071  2090  2121  2128
  2137  2151  2158  2195  2202  2211  2227  2310  2331  2336  2342  2348
  2380  2381  2394  2415  2418  2512  2515  2516  2545  2553  2561  2575
  2602  2634  2685  2688  2696  26

In [14]:
import joblib
joblib.dump(kr_model, '/content/drive/MyDrive/dacon/model/feature_model/kr_model.pkl')
joblib.dump(theilsen_model, '/content/drive/MyDrive/dacon/model/feature_model/theilsen_model.pkl')

['/content/drive/MyDrive/dacon/model/feature_model/theilsen_model.pkl']

In [15]:
xgb_param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}
catboost_param_grid = {
    'iterations': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}


In [7]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 24.8 MB/s eta 0:00:00


In [17]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_grid,
    n_iter=20,
    scoring=custom_scorer,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# 랜덤 서치 실행
xgb_random_search.fit(X, y)

# 최적 파라미터와 RMSE 출력
print("Best parameters found: ", xgb_random_search.best_params_)
best_xgb_model = xgb_random_search.best_estimator_
xgb_predictions = best_xgb_model.predict(X_test)
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
print(f"XGBoost RMSE: {xgb_rmse}")

joblib.dump(best_xgb_model, '/content/drive/MyDrive/dacon/model/feature_model/xgb_model.pkl')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters found:  {'subsample': 0.6, 'n_estimators': 400, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
XGBoost RMSE: 1.5856491327285767


['/content/drive/MyDrive/dacon/model/feature_model/xgb_model.pkl']

In [18]:
from catboost import CatBoostRegressor

# CatBoost 모델과 RandomizedSearchCV 설정
catboost_model = CatBoostRegressor(silent=True)
catboost_random_search = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=catboost_param_grid,
    n_iter=20,
    scoring=custom_scorer,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# 랜덤 서치 실행
catboost_random_search.fit(X, y)

# 최적 파라미터와 RMSE 출력
print("Best parameters found: ", catboost_random_search.best_params_)
best_catboost_model = catboost_random_search.best_estimator_
catboost_predictions = best_catboost_model.predict(X_test)

catboost_rmse = mean_squared_error(y_test, catboost_predictions, squared=False)
print(f"CatBoost RMSE: {catboost_rmse}")

joblib.dump(best_catboost_model, '/content/drive/MyDrive/dacon/model/feature_model/catboost_model.pkl')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best parameters found:  {'learning_rate': 0.01, 'l2_leaf_reg': 9, 'iterations': 400, 'depth': 10}
CatBoost RMSE: 1.5838857219642892


['/content/drive/MyDrive/dacon/model/feature_model/catboost_model.pkl']

In [19]:
# from sklearn.ensemble import VotingRegressor


# # 앙상블 모델 초기화
# model = VotingRegressor(estimators=[('th', theilsen_model), ('kr', kr_model)],weights=[1, 1])

# # 모델 학습


# # # RandomizedSearchCV 설정
# # random_search = RandomizedSearchCV(
# #     estimator=model,
# #     param_distributions=vr_params,
# #     n_iter=30,  # 시도할 하이퍼파라미터 조합의 수
# #     cv=3,  # 3-fold cross-validation
# #     scoring='neg_mean_squared_error',  # 평가 척도
# #     random_state=42,  # 결과 재현을 위한 랜덤 시드
# #     n_jobs=-1,  # 병렬처리
# #     verbose=2  # 진행 상황 출력
# # )

# # # 모델 훈련 및 하이퍼파라미터 튜닝
# # random_search.fit(X, y)

# # # 최적의 모델 및 하이퍼파라미터 확인
# # vr_model = random_search.best_estimator_
# # vr_params = random_search.best_params_

# model.fit(X, y)
# vr_model=model

# y_pred=vr_model.predict(X_test)

# rmse = mean_squared_error(y_test, y_pred, squared=False)


# print("KernelRidge:" , rmse)

In [20]:

# y_pred = model.predict(X_test)

# # RMSE 계산
# rmse = mean_squared_error(y_test, y_pred, squared=False)

# rmse